昨天我们从Excel总表中拆分了各部门的年假数据，今天就来批量发送邮件。要求是将各部门的Excel作为附件发给该部门的人员。邮件的标题为“XX部年假情况”，邮件内容也要体现“XX部”。

因此，我们先要将各部门的收件地址准备好，可以写入一个Excel表格中，然后在发送邮件之前读取。我们用`openpyxl`模块来读取邮件地址Excel表的内容。新建一个字典`address`，将部门作为键，收件人和抄送人地址作为值。抄送地址若有多个，需要用英文中的分号隔开。结果如下。

In [27]:
from openpyxl import load_workbook
wb = load_workbook("邮件地址.xlsx")
ws = wb.active
address = {}
for i in range(2,ws.max_row+1):
    dept = ws["A"+ str(i)].value
    to_add = ws["B"+ str(i)].value
    cc_add = ws["C"+ str(i)].value
    address[dept] = [to_add,cc_add]

address

{'工程部': ['xxx1@qq.com', 'yyy@163.com;zzz@21cn.com'],
 '生产部': ['xxx2@qq.com', 'yyy@163.com;zzz@21cn.com'],
 '人事部': ['xxx3@qq.com', 'yyy@163.com;zzz@21cn.com'],
 '总务部': ['xxx4@qq.com', 'yyy@163.com;zzz@21cn.com'],
 '船务部': ['xxx5@qq.com', 'yyy@163.com;zzz@21cn.com'],
 '关务部': ['xxx6@qq.com', 'yyy@163.com;zzz@21cn.com'],
 '品质部': ['xxx7@qq.com', 'yyy@163.com;zzz@21cn.com'],
 '物料部': ['xxx8@qq.com', 'yyy@163.com;zzz@21cn.com'],
 '研发部': ['xxx9@qq.com', 'yyy@163.com;zzz@21cn.com'],
 '采购部': ['xxx10@qq.com', 'yyy@163.com;zzz@21cn.com']}

地址获取完成后，就可以准备发送邮件了。由于不同的部门需要单独发送一次邮件，所以这里定义了一个邮件发送函数`Send_mail`，以便重复调用。函数中需要传入4个参数，分别是`to_add`(对应收件人地址）、`cc_add`（对应抄送地址）、`file_path`（对应附件路径）、`dept`（对应部门名称）。然后使用`win32.Dispatch('Outlook.Application')`调用windows outlook应用，通过`outlook.CreateItem(0)`创建邮件等操作。这些都是固定写法，直接复制，按需调整即可。需要说明的是，如果有多个附加需要添加，可建一个for循环，多次调用`Attachments.Add()`即可。在运行程序之前，保持outlook软件处于打开状态，邮件就可以自动发送。发送完后，可以在“已发送邮件”里看到。若程序运行的时候，outlook软件没打开，则只会将邮件放入发件箱，待软件打开后才会发送。

In [18]:
import win32com.client as win32
def Send_mail(to_add, cc_add,file_path,dept):
    '''
    传入参数说明：
    to_add,收件人地址
    cc_add,抄送地址
    file_path,附件路径
    dept,部门名称
    '''
    outlook = win32.Dispatch('Outlook.Application') # 调用windows outlook应用
    mail = outlook.CreateItem(0) # 创建邮件
    mail.to = to_add #收件人
    mail.cc = cc_add #抄送人
    mail.Subject = "{}年假情况".format(dept) #主题
    mail.Attachments.Add(file_path) #添加附件。若有多个附件，则多调用几次即可
    mail.Body = '''Dear All,\n这是{}的年假情况，请查收！谢谢。\n\nBest regards!\n人事部 小李'''.format(dept)#正文内容
    mail.Send() #发送邮件

发送邮件的函数定义好后，就可以调用它进行邮件的精准投递了。此处，附件的路径必须是绝对路径，不然会找不到文件。所以此处使用`os.getcwd()`获取工作路径，并按部门名称进行拼接，以确保对应关系。

In [26]:
#发送邮件到各部门
for dept in address.keys():    
    to_add = address[dept][0]
    cc_add = address[dept][1]
    file_path = os.getcwd() + '\\年假_按部门\\年假情况_{}.xlsx'.format(dept)    
    Send_mail(to_add, cc_add,file_path,dept)  
    
print("邮件发送完成。")

邮件发送完成。


邮件在几秒内发送完毕。在outlook界面的发件箱可以看到发送情况。也能在接收邮箱收到邮件。仅此，喝杯茶去，不谢！
![](images\结果.png)
![](images\result1.png)